In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as  plt
import tensorflow as tf
tf.__version__

F:\Anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'2.1.0'

In [2]:
df=pd.read_csv("Fake NEWS Dataset/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.dropna(inplace=True)
df.shape

(18285, 5)

In [5]:
X=df.drop("label", axis=1)
y=df['label']

In [6]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [7]:
vocabulary_size=5000

## One-Hot Representation

In [8]:
messages=X.copy()
messages.reset_index(inplace=True)

In [9]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [10]:
ps=PorterStemmer()
corpus=[]

In [11]:
for i in range(0, len(messages)):
    
#     print(i, end=' ')
    review=re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower().split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words("english")]
    review=' '.join(review)
    corpus.append(review)

In [12]:
one_hot_repr=[one_hot(word, vocabulary_size) for word in corpus]

In [13]:
## setting sentenses as of same length using pad_sequences

sent_length=20
embedded_docs=pad_sequences(one_hot_repr, padding='pre', maxlen=sent_length) ###padding-> post, pre
print(embedded_docs)

[[   0    0    0 ... 4526 1612 2113]
 [   0    0    0 ... 2912 4887 1463]
 [   0    0    0 ...  684  504 2028]
 ...
 [   0    0    0 ...  303 2053 3673]
 [   0    0    0 ... 4219  183 4571]
 [   0    0    0 ... 1510 2154 3136]]


In [14]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  726,
       2523, 2553, 1917, 3401, 2990, 3746, 4526, 1612, 2113])

In [15]:
embedding_vector_feature=40 ###Output dim
model=Sequential()

In [16]:
model.add(Embedding(vocabulary_size, embedding_vector_feature, input_length=sent_length))

In [17]:
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [19]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
(X_final.shape, y_final.shape)

((18285, 20), (18285,))

In [21]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test=tts(X_final, y_final, test_size=0.1, random_state=42)

### Model Training

In [22]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Train on 16456 samples, validate on 1829 samples
Epoch 1/10
16456/16456 [==============================] - 12s 720us/sample - loss: 0.3034 - accuracy: 0.8696 - val_loss: 0.2083 - val_accuracy: 0.9125
Epoch 2/10
16456/16456 [==============================] - 8s 515us/sample - loss: 0.1423 - accuracy: 0.9431 - val_loss: 0.2155 - val_accuracy: 0.9120
Epoch 3/10
16456/16456 [==============================] - 8s 503us/sample - loss: 0.1053 - accuracy: 0.9606 - val_loss: 0.2586 - val_accuracy: 0.9169
Epoch 4/10
16456/16456 [==============================] - 8s 504us/sample - loss: 0.0784 - accuracy: 0.9709 - val_loss: 0.2819 - val_accuracy: 0.9153
Epoch 5/10
16456/16456 [==============================] - 8s 505us/sample - loss: 0.0513 - accuracy: 0.9833 - val_loss: 0.3404 - val_accuracy: 0.9109
Epoch 6/10
16456/16456 [==============================] - 8s 495us/sample - loss: 0.0353 - accuracy: 0.9885 - val_loss: 0.4139 - val_accuracy: 0.9060
Epoch 7/10
16456/16456 [==========================

In [23]:
y_pred=model.predict_classes(X_test)
y_pred[:10]

array([[1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

### Performance Matrix

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [25]:
confusion_matrix(y_test, y_pred)

array([[922, 114],
       [ 56, 737]], dtype=int64)

In [26]:
accuracy_score(y_test, y_pred)

0.9070530344450519